In [1]:
import json
from collections import Counter
import pickle
import numpy as np
import pandas as pd
import os
import sys
import argparse
from os.path import dirname, realpath

import hashlib
import datetime
import datetime as datetime
sys.path.append("./src/")
#from disrisknet.utils.parsing import parse_args
import torch
from torch import nn, optim
from torch.nn import functional as F

In [2]:
sys.path.append(dirname(dirname(realpath(os.getcwd()))))


In [3]:
src_path = "G:\\FillmoreCancerData\\markhe\\VTERisk" 
src_path2 = "G:\\FillmoreCancerData\\markhe\\VTERisk - Copy" 

def md5(key):
    return hashlib.md5(repr(key).encode()).hexdigest()

testDF = pd.read_csv(os.path.join(src_path2, 'Notebooks/Find/fixed_dx.csv'))
pat_ids = (testDF['patient_id'] ).astype(int)

testDF['pids'] = pat_ids.apply(md5)

In [4]:
logpath = os.path.join(src_path ,'logs_transformer_vte/H/11_7/', "d93e7956f15305ad23c78d6884ca2816.results.test_preds")

In [5]:
outpath = os.path.join(src_path ,'Notebooks//PostRun_Analysis//output/Multi', "J10DX.csv")


In [6]:
tdf = pd.DataFrame({'patient_id': (testDF['patient_id'] ),    
                    'pids': (testDF['pids'] ),    
                  'dob': (testDF['dob'] ),  
                  'outcome_date': (testDF['outcome_date'] ), 
                  'obs_time_end': (testDF['obs_time_end'] ),  
                  'index_date': (testDF['index_date'] ),  
                  'diag_date': (testDF['diag_date'] ),  
                  'outcome': (testDF['outcome'])    })

with open(logpath, 'rb') as f:
    R = pickle.load(f)
    p = np.array(R['probs'])

Df = pd.DataFrame.from_dict(R)
Df['probs'] = Df['probs'].astype(float)
Df['exams'] = Df['exams'].astype(int)
M = pd.merge(tdf, Df)
 

In [7]:
Df['golds']

0        False
1        False
2        False
3        False
4        False
         ...  
82988    False
82989    False
82990    False
82991    False
82992    False
Name: golds, Length: 82993, dtype: bool

In [8]:
pred = np.array((Df['probs']))

In [9]:
pred

array([0.2147285 , 0.12524192, 0.07535204, ..., 0.14071263, 0.0988251 ,
       0.11033526])

In [10]:
len(Df)

82993

In [39]:
x0= np.zeros( shape = [len(Df),2])

In [43]:
x0[:,0] = Df['probs'].to_numpy()

In [44]:
x0

array([[0.2147285 , 0.        ],
       [0.12524192, 0.        ],
       [0.07535204, 0.        ],
       ...,
       [0.14071263, 0.        ],
       [0.0988251 , 0.        ],
       [0.11033526, 0.        ]])

In [24]:
#m_pred = np.array(Df['probs'].to_numpy(), np.zeros(len(Df)))

TypeError: Cannot construct a dtype from an array

In [69]:
y = np.array(Df['golds'])

In [45]:
def _row_max_normalization(data: np.ndarray) -> np.ndarray:
    '''Normalise the output by subtracting
       the per-row maximum element.
    '''
    row_max: np.ndarray = np.max(data,   axis = 1,  keepdims = True )
  #  row_max: np.ndarray = data 
    
    return data 

In [47]:
_row_max_normalization(x0)

array([[0.2147285 , 0.        ],
       [0.12524192, 0.        ],
       [0.07535204, 0.        ],
       ...,
       [0.14071263, 0.        ],
       [0.0988251 , 0.        ],
       [0.11033526, 0.        ]])

In [48]:
def _softmax_T(predictions: np.ndarray, 
               temperature: float,
              ) -> np.ndarray:
    '''Softmax function scaled by the
       inverse temperature.
    '''
    
    softmax_T_output: np.ndarray = predictions
    softmax_T_output = _row_max_normalization(softmax_T_output)  
    softmax_T_output /= temperature  
    softmax_T_output = softmax(softmax_T_output, 
                               axis = 1
                              )
    softmax_T_output = softmax_T_output.astype(dtype = predictions.dtype)
    
    return softmax_T_output

def _exp_T(predictions: np.ndarray, 
           temperature: float
          ) -> np.ndarray:
    '''Scale by inverse temperature,
       and then apply the nature
       exponential function
    '''
    
    exp_T_output: np.ndarray = predictions
    exp_T_output = _row_max_normalization(exp_T_output)
    exp_T_output /= temperature
    exp_T_output = np.exp(exp_T_output)
    
    return exp_T_output 

def temperature_scaling(predictions: np.ndarray, 
                         labels: np.ndarray, 
                         initial_temperature: float
                        ) -> float:
    
    def negative_log_likelihood(temperature: float):
        '''Negative Log Likelihood Loss with respect
           to Temperature
        '''
        
        # Losses
        losses: np.ndarray = _softmax_T(predictions, 
                                          temperature
                                         )
            
        # Select the probability of the correct class
        losses = losses[np.arange(losses.shape[0]), 
                        labels
                       ]
        
        losses = np.log(losses)
        
        # Derivates with respect to Temperature
        exp_T: np.ndarray = _exp_T(predictions, temperature)
        exp_T_sum = exp_T.sum(axis = 1)
        
        term_1: np.ndarray = _row_max_normalization(predictions)
        term_1 /= temperature ** 2
        term_1 = - term_1[np.arange(term_1.shape[0]), 
                          labels
                         ]
        term_1 *= exp_T_sum
                
        term_2: np.ndarray = _row_max_normalization(predictions)
        term_2 /= temperature ** 2
        term_2 = _row_max_normalization(term_2)
        term_2 *= exp_T
        term_2 = term_2.sum(axis = 1)
        
        dL_dts: np.ndarray = (term_1 + term_2) / exp_T_sum
            
        # print(f"{-losses.sum() = },  {-dL_dts.sum() = }")
            
        return -losses.sum(),  -dL_dts.sum()
    
    temperature_minimizer: minimize = minimize(negative_log_likelihood, 
                                               initial_temperature, 
                                               method = "L-BFGS-B",
                                               jac = True,
                                               options = {"gtol": 1e-6,
                                                           "ftol": 64 * np.finfo(float).eps,
                                                         }
                                              )
        
    return temperature_minimizer.x[0]

In [49]:
from scipy.optimize import minimize
from scipy.special import softmax

In [51]:
y

array([False, False, False, ..., False, False, False])

In [52]:
np.shape(y)

(82993,)

In [53]:
np.shape(x0)

(82993, 2)

In [75]:
z = np.array(y,dtype = int)
z

array([0, 0, 0, ..., 0, 0, 0])

In [76]:
np.sum(z)

1533

In [79]:
x0

array([[0.2147285 , 0.        ],
       [0.12524192, 0.        ],
       [0.07535204, 0.        ],
       ...,
       [0.14071263, 0.        ],
       [0.0988251 , 0.        ],
       [0.11033526, 0.        ]])

In [85]:
np.shape(x0)

(82993, 2)

In [86]:
np.transpose(x0)

array([[0.2147285 , 0.12524192, 0.07535204, ..., 0.14071263, 0.0988251 ,
        0.11033526],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

In [102]:
cal2 = temperature_scaling((x0), z, .1)  
                    
    

In [103]:
cal2

0.1

In [89]:
import numpy as np
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
X, y = datasets.load_iris(return_X_y=True)
X_train, X_test, y_train, y_test = train_test_split(X, y)
SVC_classifier = SVC(probability=True)

In [90]:
SVC_classifier.fit(X_train, y_train)
svc_predictions = SVC_classifier.predict_proba(X_train)
Logistic_Regression = LogisticRegression()
Logistic_Regression.fit(X_train, y_train)
logistic_predictions = Logistic_Regression.predict_proba(X_train)
temp = temperature_scaling(svc_predictions, y_train, 1.5)

C:\Users\VHANYNHeT\AppData\Local\Temp\2\ipykernel_3996\3620560755.py:53: RuntimeWarning: divide by zero encountered in log
  losses = np.log(losses)
C:\Users\VHANYNHeT\AppData\Local\Temp\2\ipykernel_3996\3620560755.py:29: RuntimeWarning: overflow encountered in exp
  exp_T_output = np.exp(exp_T_output)
C:\Users\VHANYNHeT\AppData\Local\Temp\2\ipykernel_3996\3620560755.py:74: RuntimeWarning: invalid value encountered in add
  dL_dts: np.ndarray = (term_1 + term_2) / exp_T_sum
D:\Programs\Anaconda3\Lib\site-packages\scipy\special\_logsumexp.py:224: RuntimeWarning: invalid value encountered in subtract
  exp_x_shifted = np.exp(x - x_max)


In [91]:
temp2 = temperature_scaling(logistic_predictions, y_train, 1.5)

C:\Users\VHANYNHeT\AppData\Local\Temp\2\ipykernel_3996\3620560755.py:29: RuntimeWarning: overflow encountered in exp
  exp_T_output = np.exp(exp_T_output)
C:\Users\VHANYNHeT\AppData\Local\Temp\2\ipykernel_3996\3620560755.py:74: RuntimeWarning: invalid value encountered in add
  dL_dts: np.ndarray = (term_1 + term_2) / exp_T_sum
C:\Users\VHANYNHeT\AppData\Local\Temp\2\ipykernel_3996\3620560755.py:53: RuntimeWarning: divide by zero encountered in log
  losses = np.log(losses)


In [92]:
temp2

1.118859322679242

In [93]:
temp

1.1200943073833476

In [64]:
y_train

array([0, 0, 0, 1, 2, 0, 0, 1, 1, 2, 1, 2, 0, 2, 0, 2, 0, 1, 1, 1, 2, 0,
       0, 1, 1, 1, 2, 2, 2, 1, 0, 1, 0, 2, 2, 2, 0, 1, 1, 1, 0, 1, 2, 0,
       1, 1, 1, 1, 1, 0, 0, 2, 1, 1, 2, 0, 2, 1, 2, 0, 1, 2, 1, 2, 0, 0,
       1, 2, 0, 1, 0, 2, 0, 1, 0, 2, 1, 2, 2, 1, 2, 0, 0, 1, 0, 1, 1, 0,
       0, 0, 1, 2, 2, 0, 0, 0, 2, 0, 2, 2, 1, 0, 2, 1, 0, 2, 1, 2, 1, 2,
       2, 0])

In [66]:
np.shape(y_train)

(112,)

In [65]:
np.shape(svc_predictions)

(112, 3)

In [ ]:
svc_predictions

In [ ]:
logistic_predictions

In [95]:
y

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])

In [94]:
 logistic_predictions

array([[nan, nan, nan],
       [nan, nan, nan],
       [nan, nan, nan],
       [nan, nan, nan],
       [nan, nan, nan],
       [nan, nan, nan],
       [nan, nan, nan],
       [nan, nan, nan],
       [nan, nan, nan],
       [nan, nan, nan],
       [nan, nan, nan],
       [nan, nan, nan],
       [nan, nan, nan],
       [nan, nan, nan],
       [nan, nan, nan],
       [nan, nan, nan],
       [nan, nan, nan],
       [nan, nan, nan],
       [nan, nan, nan],
       [nan, nan, nan],
       [nan, nan, nan],
       [nan, nan, nan],
       [nan, nan, nan],
       [nan, nan, nan],
       [nan, nan, nan],
       [nan, nan, nan],
       [nan, nan, nan],
       [nan, nan, nan],
       [nan, nan, nan],
       [nan, nan, nan],
       [nan, nan, nan],
       [nan, nan, nan],
       [nan, nan, nan],
       [nan, nan, nan],
       [nan, nan, nan],
       [nan, nan, nan],
       [nan, nan, nan],
       [nan, nan, nan],
       [nan, nan, nan],
       [nan, nan, nan],
       [nan, nan, nan],
       [nan, nan

In [ ]:
 #M.to_csv(outpath) 


In [11]:
M = pd.merge(g6, testDF)
m = M[["patient_id","dx_date", "index_date", "model_date",  "outcome","outcome_date","p6" ]]
m['d']= pd.to_datetime(m['outcome_date']) - pd.to_datetime(m['model_date'])

#### THE PURPOSE OF THIS STEP IS TO ACTUALLY REPLACE THE 1's after 2 years with 0's! NEED it, in dxdate this is NOT updated
m_VE = m[ m['outcome'] ]
m_VE1 = m_VE [   (pd.to_datetime( m_VE["outcome_date"] ) -pd.to_datetime( m_VE["index_date"] )) < datetime.timedelta(730)]
m_VE0 = m_VE [   (pd.to_datetime( m_VE["outcome_date"] ) -pd.to_datetime( m_VE["index_date"] )) >= datetime.timedelta(730)]
m_VE0["outcome"] = m_VE0["outcome"].replace(True, False)  
m0= m[ ~ m['outcome'] ]

m1 = pd.concat([m_VE1, m_VE0, m0])

C:\Users\VHANYNHeT\AppData\Local\Temp\2\ipykernel_84892\383890625.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  m['d']= pd.to_datetime(m['outcome_date']) - pd.to_datetime(m['model_date'])
C:\Users\VHANYNHeT\AppData\Local\Temp\2\ipykernel_84892\383890625.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  m['d']= pd.to_datetime(m['outcome_date']) - pd.to_datetime(m['model_date'])
C:\Users\VHANYNHeT\AppData\Local\Temp\2\ipykernel_84892\383890625.py:14: SettingWithCopyWarning: 
A value is trying to be se

In [22]:
sklearn.metrics.roc_auc_score(m1['outcome'],  m1['p6'], average = 'samples')


0.5600783435183764

In [13]:
( confusion_matrix (m1['outcome'],  m1['p6']>.5) )

array([[72681,     5],
       [ 4301,     0]], dtype=int64)

In [79]:
# how many people had event within 6 months of index date

In [27]:
m1.to_csv('output/XCoh_Sen.csv') 

In [27]:
m1.to_csv('output/XCoh_Sen.csv') 

In [34]:
#m1.to_excel('output/dxMD_Sensitivity90.xlsx') 

In [35]:
statistics.quantiles( M['len180'],n=20)

KeyError: 'len180'

In [23]:
plt.hist(M['len180'])

(array([2.2715e+04, 1.8050e+03, 1.3000e+02, 2.5000e+01, 1.2000e+01,
        1.0000e+00, 1.0000e+00, 1.0000e+00, 1.0000e+00, 2.0000e+00]),
 array([1.00000e+00, 2.37810e+03, 4.75520e+03, 7.13230e+03, 9.50940e+03,
        1.18865e+04, 1.42636e+04, 1.66407e+04, 1.90178e+04, 2.13949e+04,
        2.37720e+04]),
 <BarContainer object of 10 artists>)

In [10]:
m['d']= pd.to_datetime(m['outcome_date']) - pd.to_datetime(m['model_date'])
m_VE = m[ m['outcome'] ]
m_VE1 = m_VE [   (pd.to_datetime( m_VE["outcome_date"] ) -pd.to_datetime( m_VE["index_date"] )) < datetime.timedelta(730)]
m_VE0 = m_VE [   (pd.to_datetime( m_VE["outcome_date"] ) -pd.to_datetime( m_VE["index_date"] )) >= datetime.timedelta(730)]
m_VE0["outcome"] = m_VE0["outcome"].replace(True, False)  
m0= m[ ~ m['outcome'] ]

m1 = pd.concat([m_VE1, m_VE0, m0])

C:\Users\VHANYNHeT\AppData\Local\Temp\2\ipykernel_73432\1057840980.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  m['d']= pd.to_datetime(m['outcome_date']) - pd.to_datetime(m['model_date'])
C:\Users\VHANYNHeT\AppData\Local\Temp\2\ipykernel_73432\1057840980.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  m_VE0["outcome"] = m_VE0["outcome"].replace(True, False)
